In [ ]:
import seaborn as sns #pip3 install seaborn
from datetime import date
import matplotlib.pyplot as plt
    
def errorfill(x, y, yerr, color=None, label=None, alpha_fill=0.3, ax=None):
    ax = ax if ax is not None else plt.gca()
    if color is None:
        color = ax._get_lines.get_next_color()
    if np.isscalar(yerr) or len(yerr) == len(y):
        ymin = y - yerr
        ymax = y + yerr
    elif len(yerr) == 2:
        ymin, ymax = yerr
    ax.plot(x, y, color=color, label=label)
    ax.fill_between(x, ymax, ymin, color=color, label=label, alpha=alpha_fill)
    return ax

In [ ]:
directory = '/home/data/Project_ASD/' + os.environ["USER"]+'/results'
files=['results-8x1','results-8x4'] 
filename = files[f]+'-'+'.npz'

data = np.load(directory+filename, allow_pickle=True)

search_all=data['search_all']
bestScores=data['bestScores']


In [ ]:
# Combine results from 8x4 and 8x1 results together into 8x5

bestScores=np.zeros((n_trtype,n_conds+1),dtype=int)
best_c=np.zeros((n_trtype,n_conds+1),dtype=float)
best_nPCA=np.zeros((n_trtype,n_conds+1),dtype=int)

for i in range(n_trtype):
    for j in range(n_conds):
        bestScores[i,j] = search_8x4[i,j].best_score_*100
        best_c[i,j]=search_8x4[i,j].best_params_['clf__C']
        best_nPCA[i,j]=search_8x4[i,j].best_params_['pca__n_components']
        
    bestScores[i,j+1] = search_8x1[i].best_score_*100
    best_c[i,j+1]=search_8x1[i].best_params_['clf__C']
    best_nPCA[i,j+1]=search_8x1[i].best_params_['pca__n_components']


In [ ]:
# Plot best scores and parameters 
run_identifier=clf_key_8x4+'+'+clf_key_8x1+'-pca'+str(param_grid['pca__n_components'].shape[0])+'-C'+str(param_grid['clf__C'].shape[0])+'-'+str(X.shape[1])+'features'
conds=['C1','C2','C3','C4','C all']

fig = plt.figure(figsize=(4,5))
r = sns.heatmap(bestScores, cmap='BuPu',vmin=45, vmax=85,xticklabels=conds,yticklabels=trialTypes)
r.set_title("Heatmap of best scores")
plt.savefig(save_dir+'heatmap-best-scores-'+run_identifier+'.png')

fig = plt.figure(figsize=(4,5))
r = sns.heatmap(best_nPCA, cmap='BuPu',xticklabels=conds,yticklabels=trialTypes)
r.set_title("Heatmap of best n_pca")
plt.savefig(save_dir+'heatmap-best-npc-'+run_identifier+'.png')

fig = plt.figure(figsize=(4,5))
r = sns.heatmap(best_c, cmap='BuPu',xticklabels=conds,yticklabels=trialTypes)
r.set_title("Heatmap of best c")
plt.savefig(save_dir+'heatmap-best-c-'+run_identifier+'.png')


In [ ]:
fig, ax = plt.subplots(n_trtype,n_conds,figsize=(n_conds*2,n_trtype*2),sharey=True,sharex=True)
for i in range(n_trtype):
    for j in range(n_conds):
        #print("Best parameter (CV score=%0.3f):" % search_all[i,j].best_score_)
        #print(search_all[i,j].best_params_)
        clf_key_8x4='LR-L2'
        if clf_key_8x4 in ['LR-L1','LR-L2']:
            for k in range(test_C.shape[0]):
                errorfill(test_pca,search_8x4[i,j].
                          cv_results_['mean_test_score']
                          .reshape(test_C.shape[0],test_pca.shape[0])[k].T,
                          search_8x4[i,j].cv_results_['std_test_score']
                          .reshape(test_C.shape[0],test_pca.shape[0])[k].T
                              /np.sqrt(5), ax=ax[i,j])
                ax[i,j].set_ylim(0.45,0.85)
                ax[i,j].title.set_text(trialTypes[i]+' '+'Cond'+str(j+1))
            
        else:
            errorfill(test_pca,search_8x4[i,j].cv_results_['mean_test_score'],
                      search_8x4[i,j].cv_results_['std_test_score']/np.sqrt(5),
                      ax=ax[i,j])    
        ax[i,j].set_ylim(0.45,0.85)
        
plt.savefig(save_dir+'fig-scores-8x4-'+run_8x4_identifier+'.png')


In [ ]:
fig, ax = plt.subplots(1, n_trtype, figsize=(12,3))
for i in range(n_trtype):
    #print("Best parameter (CV score=%0.3f):" % search_8x1[i].best_score_)
    #print(search_8x1[i].best_params_)
    for k in range(test_pca.shape[0]):
        errorfill(test_C,search_8x1[i].
                  cv_results_['mean_test_score']
                  .reshape(test_C.shape[0],test_pca.shape[0])[:,k],
                  search_8x1[i].cv_results_['std_test_score']
                  .reshape(test_C.shape[0],test_pca.shape[0])[:,k]/np.sqrt(5),
                  ax=ax[i])

    ax[i].set_xscale('log')
    ax[i].set_ylim(0.45,0.85)
    ax[i].title.set_text(trialTypes[i])
    
plt.savefig(save_dir+'fig-scores-8x1-'+run_8x1_identifier+'.png')


In [ ]:
for i in range(n_trtype):
    #print(np.abs(search_l1_all[i].best_estimator_['clf'].coef_))
    print(np.sum(np.abs(search_8x1[i].best_estimator_['clf'].coef_)>0.5e-2))
    plt.plot(search_8x1[i].best_estimator_['clf'].coef_.T)